# What sells a car?

We are analysts of an ads website for second hand vehicles. Hundreds of free vehicle ads are posted on our website every day. We need to study the data collected over the past few years and determine what factors influence the price of a vehicle.

# Contents <a id='back'></a>

* [Intro](#intro)
* [Stage 1. Data description](#data_review)
    * [1.1 Explore initial data](#initial_data)
* [Stage 2. Data preprocessing](#data_preprocessing)
    * [2.1 Handle missing values (if any)](#handle_miss_values)
    * [2.2 Correcting data types](#correct_data_types)
    * [2.3  Enriching data](#Enriching_data)
    * [2.4 Checking clean data](#Checking_clean_data)
* [Stage 3. Exploratory data analysis](#exploratory_data_analysis)
    * [3.1 Studying and working the outliers](#Studying_working_outliers)
    * [3.2 Studying main parameters with outliers filtration](#Studying_main_parameters_with_outliers_filtration)
    * [3.3 Ad placement period](#Ad_placement_period)
    * [3.4 Average price for each type of vehicle](#avg_price_per_vehicle)
    * [3.5 Price factor](#price_factor)
* [Conclusion](#conclusion)

## Introduction <a id='intro'></a>

In this exercise we are going to analyze a dataframe reporting several car selling publications.
The aim is to find out which features are the most correlated with the price of the vehicle.
To achieve our goal we will follow these steps:
- Collecting data
- Preparing data
- Finding one or more solution prototype
- Presenting the solution with graphics/diagrams

## Stage 1. Data description <a id='data_review'></a>

In [ ]:
# Loading all libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the data file into a DataFrame
df = pd.read_csv('vehicles_us.csv')

### 1.1 Explore initial data <a id='initial_data'></a>

The dataset contains the following fields:
- `price`
- `model_year`
-`model`
- `condition`
- `cylinders`
- `fuel` — gasoline, diesel, etc.
- `odometer` — the mileage of the vehicle when the ad was posted
- `transmission`
- `paint_color`
- `is_4wd` — if the vehicle has 4-wheel drive (Boolean type)
- `date_posted` — the date the ad was posted
- `days_listed` — from post to deletion

In [ ]:
# printing the general/summary information about the DataFrame
df.info()

In [ ]:
# printing a data sample
df.head()

Lets have a deeper view of the data

In [ ]:
df.describe()

So, we have a 51525x13 dataframe.By quickly reviewing the first lines we can see absent values in 'model_year', 'cylinders', 'odometer', 'paint_color' and 'is_4wd' column. The emptiest column is 'is_4wd'. 
Other weird thing is that values in 'model_year', 'cylinders' and 'is_4wd' are float instead of int and date columns are type object instead of datetime64

Next step will be rearrange as int values the 'model_year', 'cylinders', 'odometer' and 'is_4wd' columns. After that we dive into the columns with absent values, evaluate the possible cause and think about a way to replace them.

## Stage 2. Data preprocessing <a id='data_preprocessing'></a>

### 2.1 Handle missing values (if any) <a id='handle_miss_values'></a>

Let's work on the column 'model_year'. We'll see what values there are in it, how many null value and the best way to replace them

In [ ]:
df['model_year'].sort_values().unique()

it seems there are reasonable year in it, maybe someone very old (more than a century) but still reasonable.

In [ ]:
df['model_year'].isna().sum()

We notice that there are 3619 absent values in "model_year". To replace them I think the best thing to do evaluate the mileage of similar vehicles but before we must be sure that we have the 'odometer' value within the rows where 'model_year' is missing.

We now apply multiple conditions to filter data in "model_year" and "odometer" columns to see if we can use the "odometer" column to replace the null values in "model_year"

In [ ]:
filtro_model_year = df['model_year'].isna()
filtro_odometer = df['odometer'].isna()
comb_1 = df[filtro_model_year & filtro_odometer]

len(comb_1)

Nope, we can't use 'odometer' values to help replacing the absent values in 'model_year'. The second best option is using pricing category of similar vehicles.

Lets do the same as we did just before:

In [ ]:
filtro_model_year = df['model_year'].isna()
filtro_price = df['price'].isna()
comb_2 = df[filtro_model_year & filtro_price]

len(comb_2)

Ok, we are sure that, when we do not have the 'model_year' value in a row, we do have 'price' value. 

And now we will replace the null values in 'model_year' with the median value of the corresponding 'model'.

In [ ]:
def my_lambda(x):
    return x.fillna(x.median())

model_year_meds = df.groupby(['model'])['model_year'].transform(lambda x: my_lambda(x))


df['model_year'] = df['model_year'].fillna(model_year_meds)

To simplify things up, lets take the int of every value and verify them again

In [ ]:
df['model_year'] = df['model_year'].astype(int)
df['model_year'].sort_values().unique()

Let's skip now to the second column with null-values 'cylinders'. We proceed in a similar way as 'model_year'

In [ ]:
df['cylinders'].sort_values().unique()

And now we will replace the null values in 'cylinders' with the mean value of the corresponding 'model'.

In [ ]:
def my_lambda(y):
    return y.fillna(y.median())

cylinders_meds = df.groupby(['model'])['cylinders'].transform(lambda y: my_lambda(y))

df['cylinders'] = df['cylinders'].fillna(cylinders_meds).round().astype(int)

We verify the final values to see if the null values are gone

In [ ]:
df['cylinders'].sort_values().unique()

Ok, let's skip now to the third column with null-values 'odometer'. We proceed in a similar way as 'model_year'

In [ ]:
def my_lambda(z):
    return z.fillna(z.median())

odometer_meds = df.groupby(['condition'])['odometer'].transform(lambda z: my_lambda(z))

df['odometer'] = df['odometer'].fillna(odometer_meds)

In [ ]:
df['odometer'] = df['odometer'].round().astype(int)

Next column with null values is 'paint_color'. So lets dive into it

In [ ]:
df['paint_color'].sort_values().unique()

All colors are reasonable but there's an exception: 'custom' does not mean anything per se. So, we will use "custom" to replace null_values

In [ ]:
df['paint_color'] = df['paint_color'].fillna('custom')

Lets verify the effective replacing:

In [ ]:
df['paint_color'].sort_values().unique()

Ok! We now skip to 'is_4wd' column.
In this case, since we have a boolean and there are only 1 in the column we can assume that the missing values are zeros. Since the type of this column is float we also change it to boolean

In [ ]:
df['is_4wd'] = df['is_4wd'].fillna(0)
df['is_4wd'] = df['is_4wd'].astype('bool')

### 2.2 Correcting data types <a id='correct_data_types'></a>


After the previous operations, lets see the dataframe info 

In [ ]:
df.info()

We have now a totally filled dataframe. The only weird thing is that "date_posted" type is object instead of datetime64. Lets work it

In [ ]:
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%dT%H:%M:%S')

Now we verify:

In [ ]:
df['date_posted'].info()

Ok, well done!

### 2.3 Enriching data <a id='Enriching_data'></a>

Lets have the age of the vehicle when the post was placed and insert it in a new column "car_age"

In [ ]:
# Adding the years of the vehicle when the ad was placed
date_col = pd.DatetimeIndex(df['date_posted'])
df['car_age'] = date_col.year - df['model_year']

Lets now verify it using describe()

In [ ]:
df['car_age'].describe()

Ok, "car_age" column seems reasonable. Only thing we'll do is replacing its zeros to one and since we have to do this on zeros we should do the same for all the column.

In [ ]:
df['car_age'] = df['car_age'] + 1

And now the vehicle mileage per year

In [ ]:
# Adding average vehicle mileage per year
df['miles/year'] = (df['odometer'] / df['car_age']).round(0)
df['miles/year'] = df['miles/year'].astype('int')

It might be useful to replace the condition values with something that can be more easily manipulated. So lets do it

In [ ]:
# It might be useful to replace the condition values with something that can be more easily manipulated
def condition_index(condition):
    """
    The function returns the condition index according to the previous analysis, using the following rules:
    — index = 5 (max) for 'new' condition
    — index = 4 for 'like new' condition
    — index = 3 for 'excelent' condition
    — index = 2 for 'good' condition
    — index = 1 for 'fair' condition
    — index = 0 (min) for 'salvage' condition
    """
    
    if condition == 'new':
        return 5
    if condition == 'like new':
        return 4
    if condition == 'excellent':
        return 3
    if condition == 'good':
        return 2
    if condition == 'fair':
        return 1
    if condition == 'salvage':
        return 0

Let's create a new column 'condition_ind' using the previous function

In [ ]:
df['condition_ind'] = df['condition'].map(condition_index)

Lets find now the day of the week when the vehicle has been posted and add it in a new column "day_posted"

In [ ]:
df['day_posted'] = df['date_posted'].dt.weekday

The same with month:

In [ ]:
df['month_posted'] = df['date_posted'].dt.month

### 2.4 Checking clean data <a id='Checking_clean_data'></a>

Since the data is ready for analysis, we take another quick look at it. It is useful to see what exactly we are going to use for analysis.

In [ ]:
# printing the general/summary information about the DataFrame
df.info()

In [ ]:
# printing a data sample
df.head()

## Stage 3. Exploratory data analysis <a id='exploratory_data_analysis'></a>

We are now going to study the parameters listed below, paying special attention to how outliers affect the shape histograms.

Los parámetros son:
- Precio
- Los años del vehículo cuando el anuncio se colocó
- Millaje
- Número de cilindros
- Estado

In [ ]:
mean_price = df.pivot_table(index='model', values='price')
mean_price.hist(edgecolor = 'black')
plt.title('Mean price related with the number of models')
plt.xlabel('price ($)')
plt.ylabel('number of models')
plt.show()

In the last graph, the horizontal axis tells us the price, while the vertical axis tells us the number of each model (100 in total) with average prices in each interval.We see that there are two little bars detached. This may be due to outliers.

Lets now do the same things but this time in the y-axis there'll be the brute number of vehicles.

In [ ]:
df.hist('price', bins=100, edgecolor = 'black')
plt.xlim(0, 80000)
plt.title('Mean price related with the number of cars')
plt.xlabel('price ($)')
plt.ylabel('number of vehicles')
plt.show()

In this case, is quite hard to recognize any outliers

Lets repeat the process with the car_age column

In [ ]:
mean_car_age = df.pivot_table(index='model', values='car_age')
mean_car_age.hist(edgecolor = 'black')
plt.title('Mean age related with the number of models')
plt.xlabel('age (years)')
plt.ylabel('number of models')
plt.show()

Im this last there's definitely some outliers due to the little bar on the right

In [ ]:
df.hist('car_age', bins=40, edgecolor = 'black')
plt.title('Mean age related with the number of cars')
plt.xlabel('age')
plt.ylabel('number of vehicles')
plt.show()

Lets repeat the process with the odometer column

In [ ]:
median_mileage = df.pivot_table(index='model', values='odometer', aggfunc='median')
median_mileage.hist(edgecolor = 'black')
plt.title('Median mileage related with the number of models')
plt.xlabel('mileage (miles)')
plt.ylabel('number of models')
plt.show()

It seems there are no outliers in this last histograms 

In [ ]:
df.hist('odometer', bins=40, edgecolor = 'black')
plt.title('mileage related with the number of cars')
plt.xlabel('mileage')
plt.ylabel('number of vehicles')
plt.show()

Lets analyze now the cylinder column. In this case, we use another method because there are not engines with floating number of cylinders so it is better that we avoid dynamic table.

In [ ]:
df['cylinders'].value_counts().plot(kind='bar', grid=True)
plt.xticks(rotation=0)
plt.show()

There's not much to say about it. We see that most of cars ~17500 are 8 cylinders and less than 1000 are 7, 5 and 10 cylinders. Very few ones have 3, 9 or 12 cylinders

We do the same with the condition column

In [ ]:
df['condition'].value_counts().plot(kind='bar', grid=True)
plt.xticks(rotation=45)
plt.show()

Ok then, the majority of cars are in excellent conditon and very few ones are in savage condition

### 3.1 Studying and working the outliers <a id='Studying_working_outliers'></a>

Next step is creating a DataFrame without outliers. The outliers can be identified after defining the lower/upper bound for the normal range of values.

Before doing that, lets see the outliers model by model in relation with price and mean age 

In [ ]:
df.boxplot(column='price', by='model', figsize=(50, 6))
plt.xticks(rotation=90)
plt.xlabel('car model')
plt.show()

Cool! we have definitely a better perspective now

If the analysis model by model is not important for us, then the following boxplot is more appropriate

In [ ]:
df.boxplot(column='price')
plt.title('Price analysis related to the number of vehicles')
plt.ylabel('number of vehicles')
plt.show()

Lets repeat the process with average car age:

In [ ]:
df.boxplot(column='car_age', by='model', figsize=(50, 6))
plt.xticks(rotation=90)
plt.xlabel('car model')
plt.show()

If the analysis model by model is not important for us, then the following boxplot is more appropriate

In [ ]:
df.boxplot(column='car_age')
plt.title('Age analysis related to the number of vehicles')
plt.ylabel('number of vehicles')
plt.show()

Now we are going to work on the outliers. Lets begin to the ones related to price

Since we are working model by model, first step is create a table that contains lower quartile, upper quartile, median, upper and lower bound for every car model.

In [ ]:
table_price = df.groupby("model")["price"].describe()
table_price["upper_bound"] = table_price["75%"] + 1.5*(table_price["75%"] - table_price["25%"])
table_price["lower_bound"] = table_price["25%"] - 1.5*(table_price["75%"] - table_price["25%"])
table_price

Now we are going to filter the original dataframe with the values just found 

To do so, we initialize a new empty dataframe (df_price_filtered) and fill it row by row using a for cycle

In [ ]:
df_price_filtered = pd.DataFrame()

for model in df['model'].unique():
    new_df =df.loc[ (df['model'] == model) & (df['price'] < table_price.loc[model, "upper_bound"]) & (df['price'] > table_price.loc[model, "lower_bound"])]
    df_price_filtered = df_price_filtered.append(new_df)

We have now a one time (by the price) filtered dataframe. Lets see the size of this new dataframe

In [ ]:
df_price_filtered.shape

So 1632 rows have been filtered

Now, lets take the price filtered dataframe and filter it by the average age

In [ ]:
table_age = df_price_filtered.groupby("model")["car_age"].describe()
table_age["upper_bound"] = table_age["75%"] + 1.5*(table_age["75%"] - table_age["25%"])
table_age["lower_bound"] = table_age["25%"] - 1.5*(table_age["75%"] - table_age["25%"])
table_age

Now we are going to filter the price filtered dataframe (df_price_filtered) with the values just found 

To do so, we initialize a new empty dataframe (df_price_age_filtered) and fill it row by row using a for cycle

In [ ]:
df_price_age_filtered = pd.DataFrame()

for model in df_price_filtered['model'].unique():
    new_df2 =df_price_filtered.loc[ (df_price_filtered['model'] == model) & (df_price_filtered['car_age'] < table_age.loc[model, "upper_bound"]) & (df_price_filtered['car_age'] > table_age.loc[model, "lower_bound"])]
    df_price_age_filtered = df_price_age_filtered.append(new_df2)

We have now a two times (by the price and age) filtered dataframe. Lets see the size of this new dataframe

In [ ]:
df_price_age_filtered.shape

Other 1000 lines have been filtered and so we have (1632 + 1000) 2632 rows less than the original dataframe

Lets now plot the new filtered dataframes, the first two taking into account the model by model and the others with no discrimination

In [ ]:
df_price_age_filtered.boxplot(column='price', by='model', figsize=(50, 6))
plt.xticks(rotation=90)
plt.title('filtered dataframe')
plt.xlabel('car model')
plt.show()

Of course there are still outliers but they are definetly less

In [ ]:
df_price_age_filtered.boxplot(column='car_age', by='model', figsize=(50, 6))
plt.xticks(rotation=90)
plt.title('filtered dataframe')
plt.xlabel('car model')
plt.show()

The difference with the original dataframe is more remarkable as the other ones

And now lets see boxplots  with no model distinction

In [ ]:
df_price_age_filtered.boxplot(column='price')
plt.title('Price analysis related to the number of vehicles')
plt.ylabel('number of vehicles')
plt.show()

The y range has decreased substantially and this is a good sign of filtration

In [ ]:
df_price_age_filtered.boxplot(column='car_age')
plt.title('Age analysis related to the number of vehicles')
plt.ylabel('number of vehicles')
plt.show()

Here as well we can appreciate a good filtration because the outliers are much less.

### 3.2 Studying main parameters with outliers filtration <a id='Studying_main_parameters_with_outliers_filtration'></a>

Now we use the filtered dataframe to create new histograms and compare them with the original

In [ ]:
mean_price_filt = df_price_age_filtered.pivot_table(index='model', values='price')
plt.hist(mean_price, edgecolor = 'black', label= 'original dataframe')
plt.hist(mean_price_filt, edgecolor = 'black', alpha = 0.65, label= 'filtered dataframe')
plt.legend(loc='upper right')
plt.xlabel('price ($)')
plt.show()

Cool ! The price outliers are disapperead 

Lets do the same with mean age as well:

In [ ]:
mean_car_age_filt = df_price_age_filtered.pivot_table(index='model', values='car_age')
plt.hist(mean_car_age, edgecolor = 'black', label= 'original dataframe')
plt.hist(mean_car_age_filt, edgecolor = 'black', alpha = 0.65, label= 'filtered dataframe')
plt.legend(loc='upper right')
plt.xlabel('age (years)')
plt.show()

For an unknown reason, also the filterd dataframe present an outliers. It may be the case to filter the outliers once more but in this case we keep it like this.

### 3.3 Ad placement period <a id='Ad_placement_period'></a>

Lets see model by model how many days cars have been posted taking into account mean and median value and have them arranged in descending order.

In [ ]:
time_posted_by_model = df_price_age_filtered.groupby('model').agg({'days_listed': ['median', 'mean']}).astype('int')
time_posted_by_model.columns = ['median', 'mean']
time_posted_by_model.sort_values(by='median', ascending=False, inplace=True)
time_posted_by_model

Lets see now the period of habitual placement of an ad.

In [ ]:
df_price_age_filtered.groupby('day_posted')['day_posted'].count().sort_values(ascending=False)

In [ ]:
df_price_age_filtered.groupby('month_posted')['month_posted'].count().sort_values(ascending=False)

This means that most of the publication occur on Thursday and October

Now we are going to determine when posts are quickly removed and when they are running for a long time

In [ ]:
table_type = df_price_age_filtered.groupby("type")["days_listed"].describe()
table_type.sort_values(by='50%', ascending=False, inplace=True)
table_type

So, taking into account the median value, bus is the type of vehicle that takes longer to be sold whereas van take less. Anyway we can detect some atipical max values well beyond the mean and median values

Without considering the vehicle type, lets see how many days_listed could be considered abnormally long and how many publication days could be considered abnormally short.

In [ ]:
df_price_age_filtered['days_listed'].describe()

In [ ]:
df_price_age_filtered.boxplot(column='days_listed')
plt.title("'days_listed' related to the number of vehicles")
plt.ylabel('number of vehicles')
plt.show()

So, generally speaking, all the values above 100 days could be considered outliers.

### 3.4 Average price for each type of vehicle <a id='avg_price_per_vehicle'></a>

Lets now consider the average price for each type of vehicle.

In [ ]:
price_by_type = df_price_age_filtered.groupby('type').agg({'price': ['median', 'mean']})
price_by_type.columns = ['median', 'mean price']
price_by_type.sort_values(by='median', ascending=False, inplace=True)
price_by_type

The type of vehicles sold for the highest price are the trucks whereas hatchback are sold with the lowest price

Lets now draw a graph showing the number of advertisements on each type of vehicle.

In [ ]:
df_price_age_filtered['type'].value_counts().plot(kind='bar', grid=True)
plt.xticks(rotation=65)
plt.show()

SUV and truck type are the most posted

### 3.5 Price factor <a id='price_factor'></a>

Lets see now which factors impact the price the most. To do that we take each of the most popular types of vehicle we discovered in the previous step (truck and suv) and see if the price depends on age, mileage, condition, transmission type, and color.

First step to do so is create a new dataframe that take into account only Truck type vehicles

In [ ]:
truck_filter = df_price_age_filtered['type'] == 'truck'
truck = df_price_age_filtered[truck_filter]

Since the last dataframe has 18 column but we need just need 6 we have to eliminate some columns

In [ ]:
new_truck = truck.drop(['model_year', 'model', 'condition', 'cylinders', 'fuel', 'is_4wd', 'date_posted', 'days_listed', 'miles/year', 'day_posted', 'month_posted'], axis='columns')

Lets use now the correlation matrix:

In [ ]:
new_truck.corr()

As we could imagine, price and milage are inversely proportional. Same for price and age while, of course the price raises with better conditions

To analyze the correlation with categorical variables (transmission type and color) we use boxplot

In [ ]:
new_truck.boxplot(column='price', by='paint_color')
plt.xticks(rotation=60)
plt.xlabel('color')
plt.ylabel('price ($)')
plt.show()

It seems that black vehicles are generally more expensive

In [ ]:
new_truck.boxplot(column='price', by='transmission')
plt.xticks(rotation=60)
plt.xlabel('transmission')
plt.ylabel('price ($)')
plt.show()

Of course, automatic gearbox vehicles are more expensive than manual

Lets repeat for the second most posted type of vehicle: SUV

In [ ]:
suv_filter = df_price_age_filtered['type'] == 'SUV'
suv = df_price_age_filtered[suv_filter]

Since the last dataframe has 18 column but we need just need 6 we have to eliminate some columns

In [ ]:
new_suv = suv.drop(['model_year', 'model', 'condition', 'cylinders', 'fuel', 'is_4wd', 'date_posted', 'days_listed', 'miles/year', 'day_posted', 'month_posted'], axis='columns')

Lets use now the correlation matrix:

In [ ]:
new_suv.corr()

The behaviour is pretty similar as the truck type

To analyze the correlation with categorical variables (transmission type and color) we use boxplot

In [ ]:

new_suv.boxplot(column='price', by='paint_color')
plt.xticks(rotation=60)
plt.xlabel('color')
plt.ylabel('price ($)')
plt.show()

In this case, orange seems to be the most expensive color

In [ ]:
new_suv.boxplot(column='price', by='transmission')
plt.xticks(rotation=60)
plt.xlabel('transmission')
plt.ylabel('price ($)')
plt.show()

It seems weird but for SUV vehicles, manual gearbox is more expensive

Lets focus now on the scatter plot diagrams

In [ ]:
pd.plotting.scatter_matrix(new_truck, figsize=(8, 8)) 
plt.show()

Each cell in this 4x4 grid shows a relationship between a pair of parameters. Let's take a look:

- The top left cell shows the connection between price and price. This is not a scatterplot, but an ordinary histogram showing the distribution of price. We can see that normally cars costs between 8000 and 20000$.
- The cell below shows the connection between price and mileage. The natural trend is that the less mileage the vehicle has the higher it will cost.
- Price and age: similarly as the cell above, the trend is that the less is the age higher it will cost and viceversa.
- The bottom left cell of the scatterplot shows that vehicles in better conditions are generally more expensive. 
- The condition histogram (bottom right) shows that the majority of vehicles are in good and excellent condition.
- The car_age histogram tells us that a large amount of data was collected on vehicles aged less than 15.
- The odometer histogram shows one peaks at 120000 miles. This is apparently the usual mileage for used cars for sale
- On the other side of the diagonal we see identical images of the same data; the axes are reversed

Lets see the second scatter plot diagram:

In [ ]:
pd.plotting.scatter_matrix(new_suv, figsize=(8, 8)) 
plt.show()

Behaviour is pretty similar as the first one.

## Conclusion <a id='conclusion'></a>

We started our project with a 51525x13 dataframe with 6 columns with null values inside them. First important thing we've done is replacing the absent values in 'model_year' with the median value of the rows with the same 'model' using a lambda function. We did the same with 'cylinder' and 'odometer'. The null values in 'paint_color' column have been replaced with 'custom', already present among the data. The 'is_4wd' columns has been filled just adding zeros to absent values. After that we changed some type format. We added some columns:
- car_age
- miles/year
- condition_ind
- day_posted
- month_posted.

After that we studied the principal parameters (price, car age, mileage, number of cylinders and condition) with histograms and bars charts. Doing so we realized that there were outliers among our data so we did operate to erase them.
Once did so we compared the old and the new filtered dataframe.
Period of posting, average price and price correlation factors have been studied as well